In [0]:
!git clone https://github.com/acmilannesta/Bert-embedding

Cloning into 'Bert-embedding'...
remote: Enumerating objects: 123, done.
remote: Counting objects: 100% (123/123), done.
remote: Compressing objects: 100% (121/121), done.
remote: Total 123 (delta 54), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (123/123), 11.49 MiB | 5.49 MiB/s, done.
Resolving deltas: 100% (54/54), done.


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os,  pandas as pd,  numpy as np, gc
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold
from hyperopt import fmin, hp, tpe, STATUS_OK, Trials
import warnings
warnings.filterwarnings("ignore")

##Extract features

In [0]:
!pip install keras_bert
!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
!unzip uncased_L-12_H-768_A-12.zip
from keras_bert import load_trained_model_from_checkpoint, Tokenizer, AdamWarmup, calc_train_steps, extract_embeddings, POOL_MAX

Using TensorFlow backend.


In [0]:
train = pd.read_csv('Bert-embedding/CDC/train.csv')
test = pd.read_csv('Bert-embedding/CDC/test.csv')

# Event weight
wt = pd.DataFrame(train.event.value_counts()/len(train)).rename(columns={'event':'weight'})
wt['event'] = wt.index
train = train.merge(wt, how='left', on='event')

# Reassign eventcode
train['event_idx'] = train.event.map({y:x for x, y in enumerate(np.sort(train.event.unique()))})

# Assign weight freqency
train['wt_freq'] = np.where(train.weight<0.01, 1, np.where(train.weight<0.05, 2, 3))

In [0]:
# train_features = extract_embeddings(
#     'uncased_L-12_H-768_A-12',
#     texts=train.text.tolist(),
#     poolings=POOL_MAX,
#     batch_size=16
# )
# train_features = pd.DataFrame(test_features)
# train_features.to_csv('/content/drive/My Drive/CDC Model/train_bert_features.csv', index=False)

In [0]:
# test_features = extract_embeddings(
#     'uncased_L-12_H-768_A-12',
#     texts=test.text.tolist(),
#     poolings=POOL_MAX,
#     batch_size=16
# )
# test_features = pd.DataFrame(test_features)
# test_features.to_csv('/content/drive/My Drive/CDC Model/test_bert_features.csv', index=False)

In [0]:
train_features = pd.read_csv('/content/drive/My Drive/CDC Model/train_bert_features.csv')
test_features = pd.read_csv('/content/drive/My Drive/CDC Model/test_bert_features.csv')

#**LGBM model**

In [0]:
!pip uninstall lightgbm -y
!pip install lightgbm --install-option=--gpu
import lightgbm as lgbm

Uninstalling lightgbm-2.2.3:
  Successfully uninstalled lightgbm-2.2.3
/usr/local/lib/python3.6/dist-packages/pip/_internal/commands/install.py:283: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)
     |████████████████████████████████| 686kB 3.4MB/s 
Skipping wheel build for lightgbm, due to binaries being disabled for it.
    Running setup.py install for lightgbm ... done


In [0]:
X = np.stack([
pd.read_csv('/content/drive/My Drive/CDC Model/oof/train_bert_uncased_oof.csv')['bert_uncased'].values,
pd.read_csv('/content/drive/My Drive/CDC Model/oof/train_robert_oof.csv')['oof_pred'].values,
pd.read_csv('/content/drive/My Drive/CDC Model/oof/train_bert_clinic_oof.csv')['oof_pred'].values], 1)
y = pd.read_csv('/content/drive/My Drive/CDC Model/oof/train_bert_uncased_oof.csv')['event_idx'].values

X_te = np.stack([
np.argmax(np.load('/content/drive/My Drive/CDC Model/oof/test_bert_uncased_oof.npy'), 1),
np.argmax(np.load('/content/drive/My Drive/CDC Model/oof/test_robert_oof.npy'), 1),
np.argmax(np.load('/content/drive/My Drive/CDC Model/oof/test_bert_clinic_oof.npy'), 1),], 1)

train = pd.read_csv('/content/drive/My Drive/CDC Model/oof/train_bert_uncased_oof.csv')

In [0]:
lgbm_param = {
        # 'num_leaves': 10,
        # 'learning_rate': 0.05,
        # 'feature_fraction': 0.05,
        # 'max_depth': -1,
        'objective': 'multiclass',
        'num_class' : 48,
        'bagging_fraction': 0.9,
        # 'verbose': -1,
        'device_type': 'gpu'
    }

def custom_eval(y_hat, data):
    y_true = data.get_label()
    y_hat = np.argmax(y_hat.reshape(-1, 48), 1)
    return 'f1', f1_score(y_true, y_hat, average='weighted'), True

kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
idx = [x for x in kf.split(train, train.wt_freq)]
oof_pred = np.zeros((len(X_te), 48))
for i, (tr_idx, val_idx) in enumerate(idx[1:], 2):
    print('Fold - {:}'.format(i))
    tr_data = lgbm.Dataset(X[tr_idx], y.ravel()[tr_idx], categorical_feature = [0, 1, 2])
    val_data = lgbm.Dataset(X[val_idx], y.ravel()[val_idx], categorical_feature = [0, 1, 2])
    clf = lgbm.train(
            lgbm_param,
            tr_data,
            num_boost_round=100000,
            verbose_eval=50,
            valid_sets=[val_data, tr_data],
            valid_names=['val', 'train'],
            early_stopping_rounds=200,
            learning_rates = lambda iter: 0.05 * (0.999 ** iter)
            # feval=custom_eval,
            )
    pred = clf.predict(X[val_idx], num_iteration=clf.best_iteration)
    score = f1_score(val_data.get_label(), np.argmax(pred, 1), average='weighted')
    print('oof - {:} - f1_score - {:.4f}'.format(i, score))
    oof_pred += clf.predict(X_te, num_iteration=clf.best_iteration) / kf.get_n_splits()

test = pd.read_csv('/content/Bert-embedding/CDC/test.csv')
test['event'] = np.argmax(oof_pred, 1)
test['event'] = test['event'].map({x:y for x, y in enumerate(np.sort(train.event.unique()))})
test.to_csv('solution.csv', index=False)
test.head(10)


In [0]:
lgbm_param = {
        'num_leaves': 7,
        # 'learning_rate': 0.05,
        'feature_fraction': 0.05,
        'max_depth': 2,
        'objective': 'multiclass',
        'num_class' : 48,
        'bagging_fraction': 0.9,
        # 'verbose': -1,
        'device_type': 'gpu'
    }

def custom_eval(y_hat, data):
    y_true = data.get_label()
    y_hat = np.argmax(y_hat.reshape(-1, 48), 1)
    return 'f1', f1_score(y_true, y_hat, average='weighted'), True

X = pd.concat([train_features, train[['age', 'sex']]], 1)
X_te = pd.concat([test_features, test[['age', 'sex']]], 1)
y = train.event_idx
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
oof_pred = np.zeros((len(test), 48))
# pred = np.load('/content/drive/My Drive/CDC Model/oof/test_bert_uncased_oof.npy')
idx = [x for x in kf.split(train, train.wt_freq)]

for i, (tr_idx, val_idx) in enumerate(idx, 1):
    print('Fold - {:}'.format(i))
    tr_data = lgbm.Dataset(X.values[tr_idx], y.ravel()[tr_idx])
    val_data = lgbm.Dataset(X.values[val_idx], y.ravel()[val_idx])
    clf = lgbm.train(
            lgbm_param,
            tr_data,
            num_boost_round=100000,
            verbose_eval=50,
            valid_sets=[val_data, tr_data],
            valid_names=['val', 'train'],
            early_stopping_rounds=100,
            learning_rate = lambda iter: 0.05 * (0.999 ** iter),
            # feval=custom_eval,
        )
    pred = clf.predict(X.values[val_idx], num_iteration=clf.best_iteration)
    score = f1_score(val_data.get_label(), np.argmax(pred, 1), average='weighted')
    print('oof - {:} - f1_score - {:.4f}'.format(i+1, score))
    oof_pred += clf.predict(X_te.values, num_iteration=clf.best_iteration) / kf.get_n_splits()
    del clf
    gc.collect()
    np.save('drive/My Drive/CDC Model/oof/test_lgbm_oof.npy', oof_pred)

14

In [0]:
# lgbm_param = {
#         'num_leaves': hp.choice('num_leaves', np.arange(2, 21)),
#         'learning_rate': hp.uniform('learning_rate', 0.005, 0.1),
#         'feature_fraction': hp.uniform('feature_fraction', 0.01, 0.5),
#         'max_depth': hp.choice('max_depth', np.arange(2, 11)),
#         'objective': 'binary',
#         # 'boosting_type': 'dart',
#         'metric': 'auc',
#         'verbose': -1,
#         'device_type': 'gpu'
#     }

# def f_lgbm(params):
#     lgbm_pred = np.zeros((len(X), ))
#     auc = np.zeros(5)
#     for i, (tr_idx, te_idx) in enumerate(StratifiedKFold(5, True, 2019).split(X, y)):
#         tr_data = lgbm.Dataset(X.values[tr_idx], y.ravel()[tr_idx], categorical_feature=cat_col)
#         te_data = lgbm.Dataset(X.values[te_idx], y.ravel()[te_idx], categorical_feature=cat_col)
#         clf = lgbm.train(params,
#                          tr_data,
#                          num_boost_round=2000,
#                          verbose_eval=200,
#                          valid_sets=[tr_data, te_data],
#                          early_stopping_rounds=False,
#                     )
#         lgbm_pred[te_idx] = clf.predict(X.values[te_idx], num_iteration=clf.best_iteration)
#         auc[i] = roc_auc_score(y.ravel()[te_idx], lgbm_pred[te_idx])
#         del clf
#         gc.collect()
#     return {'loss': -np.mean(auc).round(5), 'status': STATUS_OK}

# trials = Trials()
# lgbm_best = fmin(f_lgbm, lgbm_param, algo=tpe.suggest, rstate=np.random.RandomState(2019), max_evals=20, trials=trials)